# Viva anuncios scrapper

### Set-up

In [19]:
# Dependencies and libraries
import requests
import pandas as pd
import re
import json
import time

from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from IPython.display import clear_output


In [20]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)




====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Juanjo\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [21]:
# Loading page
url = 'https://www.vivanuncios.com.mx/s-venta-inmuebles/distrito-federal/v1c1097l1008p1'
browser.visit(url)
# Getting the page
html = browser.html


# Important: the page only loads 50 pages of results, before running the scrap check if this condition still applies

In [23]:
last_page = 50
id = 1
scrapped_data = {id:{}}
for page in range(0,last_page+1):
    # Printing progress information
    clear_output(wait=True)
    print(f'Processing page: {page}/{50}')    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    soup2 = soup.findAll('div', class_='tileV2 REAdTileV2 regular mapView')
    
    for ad in soup2:
        scrapped_data[id] = {}
        try:
            scrapped_data[id]['price'] = (ad.find(class_="ad-price").text)
        except:
            scrapped_data[id]['price'] = (0)
        try:
            scrapped_data[id]['square_meters'] = (ad.find(class_="chiplets-inline-block surface-area").text)
        except:
            scrapped_data[id]['square_meters'] = ('0')
        try:
            scrapped_data[id]['bedrooms'] = (ad.find(class_="chiplets-inline-block re-bedroom").text)
        except:
            scrapped_data[id]['bedrooms'] = ('0')
        try:
            scrapped_data[id]['bathrooms'] = (ad.find(class_="chiplets-inline-block re-bathroom").text)
        except:
            scrapped_data[id]['bathrooms'] = ('0')
        try:
            scrapped_data[id]['parking_spaces'] = (ad.find(class_="chiplets-inline-block car-parking").text)
        except:
            scrapped_data[id]['parking_spaces'] = ('0')
        try:
            scrapped_data[id]['address'] = (ad.find(class_="tile-location one-liner").text)
        except:
            scrapped_data[id]['address'] = ('0')
        try:
            scrapped_data[id]['description'] = (ad.find(class_="href-link tile-title-text").text)
        except:
            scrapped_data[id]['description'] = ('0')
        id += 1
        #browser.links.find_by_partial_href('/s-venta-inmuebles/distrito-federal/page-'+str(page+2)).click
    url = 'https://www.vivanuncios.com.mx/s-venta-inmuebles/distrito-federal/v1c1097l1008p'+str(page+2)
    browser.visit(url)
    time.sleep(2)
    

Processing page: 50/50


In [26]:
scrapped_data

{1: {'price': '2585000',
  'square_meters': '59',
  'bedrooms': '2',
  'bathrooms': '1',
  'parking_spaces': '3',
  'address': 'Olivar de los Padres',
  'description': 'Departamento Olivar de los Padres Excelente Precio para Inversion'},
 2: {'price': '4531142',
  'square_meters': '0',
  'bedrooms': '2',
  'bathrooms': '2',
  'parking_spaces': '1',
  'address': 'Nápoles',
  'description': 'DEPARTAMENTOS EN PREVENTA COL. NÁPOLES'},
 3: {'price': '2850000',
  'square_meters': '0',
  'bedrooms': '2',
  'bathrooms': '2',
  'parking_spaces': '1',
  'address': 'La Fama',
  'description': 'Departamento en venta en Barrio La Fama, Tlalpan'},
 4: {'price': '850000',
  'square_meters': '49',
  'bedrooms': '2',
  'bathrooms': '1',
  'parking_spaces': '1',
  'address': 'Mixcoatl',
  'description': 'VENDO DEPARTAMENTO  EN PLANTA BAJA A 10 MINUTOS DE METRO CONSTITUCION DE 1917'},
 5: {'price': '4500000',
  'square_meters': '0',
  'bedrooms': '2',
  'bathrooms': '1',
  'parking_spaces': '2',
  'addre

In [30]:
## Reformatting
for i in range(1,len(scrapped_data)+1):
    try:
        # Reformatting price
        scrapped_data[i]['price'] = re.sub("[^0-9]", "", scrapped_data[i]['price'])
        # Reformatting square meters
        scrapped_data[i]['square_meters'] = re.sub("[^0-9]", "", scrapped_data[i]['square_meters'])
    except:
        print ('failed some records')

failed some records


In [31]:
# Closing browser
browser.quit()

# Saving the data to a json
with open('viva_anuncios.json', 'w') as fp:
    json.dump(scrapped_data, fp, indent=4)
